<a href="https://colab.research.google.com/github/antussa2016/thesis_work/blob/master/model/thesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/antussa2016/thesis_work

Cloning into 'thesis_work'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 244 (delta 58), reused 0 (delta 0), pack-reused 147
Receiving objects: 100% (244/244), 1.00 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [145]:
df = pd.read_csv('thesis_work/dataset/final_data_for_modeling .csv')

In [15]:
df.head(1)

,twitter_id,stackoverflow_id,question_count,answer_count,reputation,answer_tag,question_tag,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,10F1,145587,13,500,74297,"['go', 'json', 'http', 'goroutine', 'struct', ...","['go', 'javascript', 'python', 'c++', 'linux',...",0.839299,0.170988,0.083984,0.047367,0.612101,0,json,0,0,1,0,0


In [146]:
df.drop(columns=['twitter_id','stackoverflow_id','question_count','reputation','answer_tag','question_tag','Extraversion'], inplace=True)

In [17]:
df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,500,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [ ]:
# Calculate min and max values for the column
min_value = df['answer_count'].min()
max_value = df['answer_count'].max()

# Apply min-max scaling to the column
df['answer_count'] = (df[['answer_count']] - min_value) / (max_value - min_value)

df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,1.0,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [147]:
import numpy as np
x= np.array(df.iloc[:,1:5])
y = np.array(df.iloc[:,8:12])


In [52]:
df.iloc[:,1:5]

,Openness,Conscientiousness,Agreeableness,Neuroticism
0,0.839299,0.170988,0.047367,0.612101
1,0.849555,0.100009,0.001654,0.392036
2,0.894954,0.124989,0.009011,0.288220
3,0.689216,0.625448,0.037679,0.223963
4,0.881779,0.042616,0.003885,0.432516
...,...,...,...,...
814,0.888007,0.739341,0.037764,0.342692
815,0.596079,0.105716,0.245279,1.000000
816,0.969066,0.055010,0.028997,0.295512
817,0.917028,0.062395,0.000687,0.461590


In [131]:
df.iloc[:,8:12]

,oop,web-pro,database,mobile
0,0,1,0,0
1,1,0,0,0
2,1,1,0,0
3,1,0,0,0
4,1,1,0,0
...,...,...,...,...
814,0,1,0,0
815,1,1,0,0
816,1,1,0,0
817,1,1,0,0


In [133]:
y = y.argmax(axis=1)

In [148]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
y_train.shape


(655,)

In [135]:
# Initialize and train an SVM classifier with hyperparameter tuning
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


predictions = best_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print("report:", report)

Accuracy: 0.6280487804878049
report:               precision    recall  f1-score   support

           0       0.63      1.00      0.77       103
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         9

    accuracy                           0.63       164
   macro avg       0.16      0.25      0.19       164
weighted avg       0.39      0.63      0.48       164



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

# **Random forest**

In [136]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf_classifier = RandomForestClassifier()
param_grid = {
   'n_estimators': [100, 200, 300],
   'max_depth': [None, 10, 20, 30],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4],
   'max_features': ['auto', 'sqrt', 'log2'],
   'bootstrap': [True, False]
}
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)
best_rf_classifier = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [138]:
from sklearn.metrics import classification_report
best_rf_classifier.fit(X_train, y_train)
y_pred = best_rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print("report:", report)

Accuracy: 0.6341463414634146
report:               precision    recall  f1-score   support

           0       0.63      0.99      0.77       103
           1       0.67      0.04      0.08        50
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         9

    accuracy                           0.63       164
   macro avg       0.33      0.26      0.21       164
weighted avg       0.60      0.63      0.51       164



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

In [ ]:
X_train.shape

(655, 4)

In [ ]:
X_test.shape

(164, 4)

In [ ]:
y_train.shape

(655, 4)

In [ ]:
y_test.shape

(164, 4)

In [149]:
from tensorflow import keras
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import ast
import json

In [150]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(4, 1)))# add 1D convolutional layer
model.add(MaxPooling1D(pool_size=2))# add max pooling layer
model.add(Flatten())# add flatten layer
model.add(Dense(64, activation='relu'))# add dense layer
model.add(Dense(4, activation='sigmoid'))# add output layer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # compile the model
model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
21/21 [==============================] - 1s 3ms/step - loss: 0.6592 - accuracy: 0.6412 
Epoch 2/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5930 - accuracy: 0.6824
Epoch 3/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5390 - accuracy: 0.6824
Epoch 4/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5237 - accuracy: 0.6824
Epoch 5/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5223 - accuracy: 0.6824
Epoch 6/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5215 - accuracy: 0.6824
Epoch 7/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5212 - accuracy: 0.6824
Epoch 8/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5215 - accuracy: 0.6824
Epoch 9/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5207 - accuracy: 0.6824
Epoch 10/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5205 - accuracy: 0.6824


In [123]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.3223 - accuracy: 0.9085


In [152]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,average='macro')
recall = recall_score(y_test, y_pred,average='macro')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)


6/6 [==============================] - 0s 2ms/step
Accuracy: 0.3170731707317073
Precision: 0.40091463414634143
Recall: 0.25297619047619047


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install scikeras

In [126]:
model = Sequential([
    keras.layers.Dense(100, input_shape=(4,), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')

])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=10,batch_size=32)

Epoch 1/10
21/21 [==============================] - 1s 12ms/step - loss: 0.6052 - accuracy: 0.8290 - val_loss: 0.5127 - val_accuracy: 0.9085
Epoch 2/10
21/21 [==============================] - 0s 4ms/step - loss: 0.5051 - accuracy: 0.8489 - val_loss: 0.4164 - val_accuracy: 0.9085
Epoch 3/10
21/21 [==============================] - 0s 4ms/step - loss: 0.4506 - accuracy: 0.8489 - val_loss: 0.3599 - val_accuracy: 0.9085
Epoch 4/10
21/21 [==============================] - 0s 4ms/step - loss: 0.4282 - accuracy: 0.8489 - val_loss: 0.3321 - val_accuracy: 0.9085
Epoch 5/10
21/21 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.8489 - val_loss: 0.3216 - val_accuracy: 0.9085
Epoch 6/10
21/21 [==============================] - 0s 4ms/step - loss: 0.4206 - accuracy: 0.8489 - val_loss: 0.3221 - val_accuracy: 0.9085
Epoch 7/10
21/21 [==============================] - 0s 4ms/step - loss: 0.4204 - accuracy: 0.8489 - val_loss: 0.3218 - val_accuracy: 0.9085
Epoch 8/10
21/21 [=

In [128]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.3204 - accuracy: 0.9085


In [127]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

6/6 [==============================] - 0s 3ms/step
Accuracy: 0.9085365853658537
Precision: 0.0
Recall: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

In [47]:
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Generate or load your data. Make sure it's suitable for LSTM input.

#Define your LSTM model
# model = Sequential([
#     LSTM(64, input_shape=(None, 4), activation='relu'),
#     LSTM(64, activation='relu'),
#     Dense(4, activation='sigmoid')  # Sigmoid activation for multi-label classification
# ])


model = Sequential()
model.add(LSTM(units=64, input_shape=(1, 4)))
model.add(Dense(units=4, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)

# Make predictions on the test set
predictions = model.predict(X_test)

# Threshold the predictions to get binary labels
threshold = 0.5
binary_predictions = (predictions > threshold).astype('int')

# Display the predicted labels and true labels for each instance
for i, (pred_labels, true_labels) in enumerate(zip(binary_predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted labels:", pred_labels)
    print("True labels:", true_labels)
    print("=" * 40)

Epoch 1/50
21/21 [==============================] - 2s 4ms/step - loss: 0.6819 - accuracy: 0.4458
Epoch 2/50
21/21 [==============================] - 0s 3ms/step - loss: 0.6569 - accuracy: 0.6824
Epoch 3/50
21/21 [==============================] - 0s 3ms/step - loss: 0.6290 - accuracy: 0.6824
Epoch 4/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5976 - accuracy: 0.6824
Epoch 5/50
21/21 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.6824
Epoch 6/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5448 - accuracy: 0.6824
Epoch 7/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5318 - accuracy: 0.6824
Epoch 8/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5256 - accuracy: 0.6824
Epoch 9/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5235 - accuracy: 0.6824
Epoch 10/50
21/21 [==============================] - 0s 4ms/step - loss: 0.5225 - accuracy: 0.6824
Epoch 11/50
21/21 [

In [ ]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.5075 - accuracy: 0.6402


In [ ]:
y.shape

(4, 4)